# Allergan - List
## Info
* Summen teils weit auseinander (nicht Rundungsdifferenz). Bsp `Krieger-Gruebel`. Total: 2128. Summentotal: 2137

In [5]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [6]:
# Read pdf into DataFrame
df = tabula.read_pdf("offenlegung-2017_ch_publikation-29-juni-2018.pdf", pages='all', lattice=True)

## Format Table

In [7]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'vorname', 'location', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty']

#Clear H\rC\rP\rs
df_export.loc[df_export.name == 'H\rC\rP\rs', 'name'] = np.NaN
df_export = df_export.replace(r"\bNa\b", np.NaN, regex=True)

#Set empty to object. otherwise value gets lost after shifting
df_export['empty'] = df_export['empty'].astype(object)

#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Remove column empty
df_export = df_export.drop(columns='empty')

#Remove empty
df_export = df_export.dropna(subset=['related_expenses', 'total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')

#Convert to Numbers
df_export = amounts_to_number(df_export)

#Set Type: Where total is 0 = hco
df_export['type'] = df_export['total'].apply(lambda x: 'hcp' if x > 0 else 'hco')

#Move HCO-Values to the right. But first add column
hco = df_export[df_export.type=='hco']
hco.insert(1, 'vorname_', np.nan)
hco.columns = ['name', 'vorname', 'location', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'empty', 'type']
hco = hco.drop(columns='empty')

#Combine Name
df_export['name'] = df_export['vorname'] + ' ' + df_export['name']

#Concat hco & hcp
df_export = pd.concat([df_export[df_export.type=='hcp'], hco])

#Remove Carination
df_export = remove_carination(df_export, " ")

#Drop vorname
df_export = df_export.drop(columns='vorname')

#Add fields
df_export = add_uci(df_export)
df_export = add_plz(df_export)
df_export = add_country(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Convert to Numbers
df_export = amounts_to_number(df_export)

#Export
export_list(df_export, 'allergan')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'amgen', True)
#write_to_excel(df_export, 'amgen')